In [1]:
#import os
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

from MLDL2024_semantic_segmentation.datasets.importDataset import Download
from MLDL2024_semantic_segmentation.datasets.importDataset import Modified_CityScapes
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
import torchvision.transforms as transforms
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *





Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 174 (delta 28), reused 1 (delta 1), pack-reused 114
Receiving objects: 100% (174/174), 127.32 KiB | 4.71 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [19]:
# TODO: Define here your training and validation loops.
import torch
import torchvision

def train(model, optimizer, train_loader, loss_fn):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        # Compute prediction and loss
        outputs =  model(inputs)
        print(outputs[0])
        loss = loss_fn(outputs[0], targets)

        # Backpropagation
        optimizer.zero_grad() # reset gradients of parameters
        loss.backward()  # backpropagate the prediction loss
        optimizer.step() # update model

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    return train_accuracy

def test(model, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    test_loss = test_loss / len(test_loader)
    test_accuracy = 100. * correct / total
    return test_accuracy

In [4]:
# Take dataset
Download('drive/MyDrive/Cityscapes.zip', '')
Modified_CityScapes('Cityscapes/Cityspaces')

Mounted at /content/drive
The zip file has been extracted correctly


In [5]:
# Setup fixed parameters
num_epochs = 5

In [6]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((512, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((512, 256)),
    transforms.ToTensor(),
])

# Create dataloader
dataset_train = CityScapes('Cityscapes/Cityspaces', split = 'train', transform = transform_image, label_transform = transform_target)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True)

dataset_val = CityScapes('Cityscapes/Cityspaces', split = 'val', transform = transform_image, label_transform = transform_target)
dataloader_val = DataLoader(dataset_train, batch_size=4, shuffle=False)

In [7]:


#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512, 256'
#torch.cuda.empty_cache()

In [8]:
# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 146MB/s]


Deeplab pretraining loading...


In [9]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [20]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

for epoch in range(num_epochs):
    train(model, optimizer, dataloader_train, loss_fn)
    test_acc = test(model, dataloader_val, loss_fn)
    print(f"Test accuracy: {test_acc:.2f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 5.06 MiB is free. Process 2744 has 14.74 GiB memory in use. Of the allocated memory 13.85 GiB is allocated by PyTorch, and 787.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
print(dataloader_train[1])


TypeError: 'DataLoader' object is not subscriptable